<a href="https://colab.research.google.com/github/martiiiss/RecommenderSystemsChallenge/blob/main/Hybrid_copy_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HYBRID MODELS

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse as sps
import matplotlib.pyplot as plt

import numpy as np
import scipy.sparse as sps
import time
import os
import importlib
from tqdm import tqdm
from numpy import linalg as la

In [2]:
np.random.seed(0)


# From github

In [3]:
!git clone https://github.com/recsyspolimi/RecSys_Course_AT_PoliMi.git

fatal: destination path 'RecSys_Course_AT_PoliMi' already exists and is not an empty directory.


In [4]:
%cd RecSys_Course_AT_PoliMi

/content/RecSys_Course_AT_PoliMi


In [5]:
!pip install lightfm

In [6]:
pip install scikit-optimize

In [7]:
pip install Cython

In [8]:
base_folder = '/content/RecSys_Course_AT_PoliMi'

for root, dirs, files in os.walk(base_folder):
    for filename in files:
        if filename.endswith('.py'):
            # Create module name from relative path, excluding ".py"
            relative_path = os.path.relpath(os.path.join(root, filename), base_folder)
            module_name = relative_path.replace(os.sep, '.')[:-3]  # Format as module path without .py

            # Import the module dynamically
            try:
                globals()[module_name] = importlib.import_module(f'{module_name}')
            except Exception as e:
                print(f"Failed to import {module_name}: {e}")

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: Movielens10M
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10126, feature occurrences: 128384, density 1.19E-03
	ICM name: ICM_year, Value range: 1.92E+03 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00
	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10106, feature occurrences: 106820, density 9.90E-04


Failed to import cython_example_SLIM_MSE: No module named 'Cython_examp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
!python3 -m pip install --upgrade setuptools

In [10]:
!python setup.py install
!pip install --upgrade setuptools cython

python3: can't open file '/content/RecSys_Course_AT_PoliMi/setup.py': [Errno 2] No such file or directory


In [11]:
# List files in the Cython_examples directory
cython_examples_dir = '/content/RecSys_Course_AT_PoliMi/Cython_examples'
print(os.listdir(cython_examples_dir))
from Cython.Build import cythonize
from setuptools import Extension

# Use a wildcard to match all .pyx files in the Cython_examples folder
extensions = cythonize(f"{cython_examples_dir}/*.pyx", compiler_directives={'language_level': 3})

# Compile the extensions
cythonize(extensions, compiler_directives={'language_level': 3})


['isPrime_slow.pyx', 'isPrime_slow.html', 'isPrime.html', 'SLIM_MSE_fastest.pyx', 'SLIM_MSE_fastest.c', 'isPrime.c', 'FunkSVD_slow.pyx', 'FunkSVD_fastest.html', 'isPrime.pyx', 'isPrime_slow.c', 'SLIM_MSE_slow.pyx', 'FunkSVD_slow.c', 'FunkSVD_slow.html', 'SLIM_MSE_slow.html', 'SLIM_MSE_fastest.html', 'isPrime_nochecks.pyx', 'FunkSVD_fastest.c', 'SLIM_MSE_slow.c', 'FunkSVD_fastest.pyx', 'isPrime_nochecks.c', 'isPrime_nochecks.html']


[<setuptools.extension.Extension('FunkSVD_fastest') at 0x7f3030d92830>,
 <setuptools.extension.Extension('FunkSVD_slow') at 0x7f3030ac2fe0>,
 <setuptools.extension.Extension('SLIM_MSE_fastest') at 0x7f3030ac31f0>,
 <setuptools.extension.Extension('SLIM_MSE_slow') at 0x7f3030ac3460>,
 <setuptools.extension.Extension('isPrime') at 0x7f3030ac3670>,
 <setuptools.extension.Extension('isPrime_nochecks') at 0x7f3030ac3880>,
 <setuptools.extension.Extension('isPrime_slow') at 0x7f3030ac3a90>]

#Useful functions

In [12]:
def split_train_test(URM_all, ICM_all, train_percentage=0.8):
    """
    Splits the URM_all and ICM_all matrices into training and testing sets,
    applying the same sampling indices to both matrices.

    :param URM_all: User-Rating Matrix (sparse)
    :param ICM_all: Item-Content Matrix (sparse)
    :param train_percentage: float, percentage of data to use for training
    :return: URM_train, URM_test, ICM_train, ICM_test
    """
    assert 0.0 <= train_percentage <= 1.0, "train_percentage must be between 0.0 and 1.0"

    # Convert URM and ICM to COO format to access their indices and values
    URM_all_coo = sps.coo_matrix(URM_all)
    ICM_all_coo = sps.coo_matrix(ICM_all)

    # Get the total number of interactions in URM_all
    total_interactions = URM_all_coo.nnz

    # Generate random indices for splitting
    indices_for_sampling = np.arange(total_interactions)
    np.random.shuffle(indices_for_sampling)

    # Split indices for training and testing
    n_train_interactions = int(total_interactions * train_percentage)
    train_indices = indices_for_sampling[:n_train_interactions]
    test_indices = indices_for_sampling[n_train_interactions:]

    # Split URM into train and test sets using the sampled indices
    URM_train = sps.coo_matrix((URM_all_coo.data[train_indices],
                                (URM_all_coo.row[train_indices], URM_all_coo.col[train_indices])),
                               shape=URM_all.shape)
    URM_test = sps.coo_matrix((URM_all_coo.data[test_indices],
                               (URM_all_coo.row[test_indices], URM_all_coo.col[test_indices])),
                              shape=URM_all.shape)

    # Split ICM into train and test sets using the same indices
    ICM_train = sps.coo_matrix((ICM_all_coo.data[train_indices],
                                (ICM_all_coo.row[train_indices], ICM_all_coo.col[train_indices])),
                               shape=ICM_all.shape)
    ICM_test = sps.coo_matrix((ICM_all_coo.data[test_indices],
                               (ICM_all_coo.row[test_indices], ICM_all_coo.col[test_indices])),
                              shape=ICM_all.shape)

    return URM_train, URM_test, ICM_train, ICM_test

# Import Data

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## URM
- user_id
- item_id
- data

In [14]:
# Carica il file CSV  -> URM
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender Systems/Challenge/data_train.csv')

# Trova il massimo user_id e item_id per dimensionare correttamente la matrice
max_user_id = data['user_id'].max()
max_item_id = data['item_id'].max()

# Crea la matrice sparsa
URM_all = csr_matrix((data['data'], (data['user_id'], data['item_id'])), shape=(max_user_id + 1, max_item_id + 1))

# Print initial URM_train matrix
print("URM_all:")
print(URM_all)

URM_all:
  (0, 0)	1.0
  (0, 2)	1.0
  (0, 120)	1.0
  (0, 128)	1.0
  (0, 211)	1.0
  (0, 232)	1.0
  (0, 282)	1.0
  (0, 453)	1.0
  (0, 458)	1.0
  (0, 491)	1.0
  (0, 675)	1.0
  (0, 711)	1.0
  (0, 760)	1.0
  (0, 884)	1.0
  (0, 950)	1.0
  (0, 1015)	1.0
  (0, 1075)	1.0
  (0, 1715)	1.0
  (0, 2548)	1.0
  (0, 2600)	1.0
  (0, 2635)	1.0
  (0, 2697)	1.0
  (0, 2698)	1.0
  (0, 2705)	1.0
  (0, 2753)	1.0
  :	:
  (35735, 37018)	1.0
  (35735, 37019)	1.0
  (35735, 37068)	1.0
  (35735, 37072)	1.0
  (35735, 37191)	1.0
  (35735, 37193)	1.0
  (35735, 37300)	1.0
  (35735, 37441)	1.0
  (35735, 37444)	1.0
  (35735, 37447)	1.0
  (35735, 37448)	1.0
  (35735, 37500)	1.0
  (35735, 37503)	1.0
  (35735, 37546)	1.0
  (35735, 37547)	1.0
  (35735, 37755)	1.0
  (35735, 37756)	1.0
  (35735, 37757)	1.0
  (35735, 37800)	1.0
  (35735, 37801)	1.0
  (35735, 37802)	1.0
  (35735, 37803)	1.0
  (35735, 37805)	1.0
  (35735, 38000)	1.0
  (35735, 38034)	1.0


## ICM matrix:
- item_id
- feature_id
- data (1 if the item is associated with the feature)


In [15]:
# Carica il file CSV -> ICM
data_ICM = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Recommender Systems/Challenge/data_ICM_metadata.csv')

# Estrai le colonne necessarie
item_ids = data_ICM['item_id'].values
feature_ids = data_ICM['feature_id'].values
data_values = data_ICM['data'].values

# Trova il numero massimo di item_id e feature_id per determinare la dimensione della matrice
num_items = item_ids.max() + 1
num_features = feature_ids.max() + 1

# Crea la matrice ICM come matrice sparsa
ICM_all = sps.csr_matrix((data_values, (item_ids, feature_ids)), shape=(num_items, num_features))

# Print initial ICM matrix
print("\nICM_matrix:")
print(ICM_all)


ICM_matrix:
  (0, 2)	1.0
  (0, 24)	1.0
  (0, 38)	1.0
  (0, 49)	1.0
  (0, 61)	1.0
  (0, 131)	1.0
  (0, 136)	1.0
  (0, 161)	1.0
  (0, 168)	1.0
  (0, 178)	1.0
  (0, 213)	1.0
  (0, 235)	1.0
  (0, 249)	1.0
  (0, 266)	1.0
  (0, 287)	1.0
  (0, 440)	1.0
  (0, 442)	1.0
  (0, 449)	1.0
  (0, 457)	1.0
  (0, 458)	1.0
  (0, 473)	1.0
  (0, 531)	1.0
  (0, 611)	1.0
  (0, 685)	1.0
  (0, 701)	1.0
  :	:
  (38119, 7430)	1.0
  (38119, 7947)	1.0
  (38119, 8974)	1.0
  (38119, 10798)	1.0
  (38119, 11497)	1.0
  (38119, 11629)	1.0
  (38119, 13586)	1.0
  (38119, 13619)	1.0
  (38119, 13886)	1.0
  (38119, 21462)	1.0
  (38119, 29427)	1.0
  (38119, 29788)	1.0
  (38119, 38157)	1.0
  (38119, 46011)	1.0
  (38119, 51718)	1.0
  (38119, 76038)	1.0
  (38119, 76039)	1.0
  (38119, 76040)	1.0
  (38120, 2)	1.0
  (38120, 271)	1.0
  (38120, 1964)	1.0
  (38120, 2012)	1.0
  (38120, 12128)	1.0
  (38120, 21462)	1.0
  (38120, 29788)	1.0


In [16]:
print(f"URM_all max row index: {URM_all.shape[0] - 1}, max col index: {URM_all.shape[1] - 1}")
print(f"ICM_all max row index: {ICM_all.shape[0] - 1}, max col index: {ICM_all.shape[1] - 1}")

URM_all max row index: 35735, max col index: 38120
ICM_all max row index: 38120, max col index: 94330


## Split dataset

In [17]:
# split the dataset into training set and validation set
URM_train, URM_test, ICM_train, ICM_test = split_train_test(URM_all, ICM_all, train_percentage=0.8)
print("\n URM_train:")
print(URM_train)
print("\n URM_test:")
print(URM_test)
print("\n ICM_train:")
print(ICM_train)
print("\n ICM_test:")
print(ICM_test)


 URM_train:
  (874, 16549)	1.0
  (11766, 19359)	1.0
  (35064, 34982)	1.0
  (9057, 21273)	1.0
  (15342, 9192)	1.0
  (25872, 15756)	1.0
  (23984, 25369)	1.0
  (9439, 10624)	1.0
  (8730, 2025)	1.0
  (2232, 3129)	1.0
  (14052, 33392)	1.0
  (22944, 20344)	1.0
  (30683, 19612)	1.0
  (23973, 6936)	1.0
  (12983, 26288)	1.0
  (5887, 19747)	1.0
  (2771, 31418)	1.0
  (5848, 1473)	1.0
  (19312, 37657)	1.0
  (22841, 32595)	1.0
  (30503, 31183)	1.0
  (10304, 37963)	1.0
  (6053, 16467)	1.0
  (14848, 15412)	1.0
  (24880, 27693)	1.0
  :	:
  (30, 16971)	1.0
  (21011, 27819)	1.0
  (6070, 17212)	1.0
  (21380, 33345)	1.0
  (17642, 30753)	1.0
  (11515, 8874)	1.0
  (14762, 4601)	1.0
  (27502, 17894)	1.0
  (20720, 2022)	1.0
  (21857, 8329)	1.0
  (13618, 6752)	1.0
  (3027, 1312)	1.0
  (5901, 11146)	1.0
  (34415, 28494)	1.0
  (24343, 26444)	1.0
  (5051, 15925)	1.0
  (19891, 8456)	1.0
  (30556, 31608)	1.0
  (17969, 6650)	1.0
  (931, 15663)	1.0
  (2620, 1914)	1.0
  (15908, 15081)	1.0
  (28706, 36872)	1.0
  (6429

## STACK:  concatenare matrici sparse verticalmente

In [18]:
stack_matrix_train = sps.vstack([URM_train, ICM_all.T])

# Model: recommender_ItemKNNCFCBF

## model: hybrid model

In [19]:
from RecSys_Course_AT_PoliMi.Recommenders import BaseRecommender
from RecSys_Course_AT_PoliMi.Recommenders.KNN import ItemKNNCBFRecommender
from RecSys_Course_AT_PoliMi.Recommenders.EASE_R import EASE_R_Recommender
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization import IALSRecommender
from RecSys_Course_AT_PoliMi.Recommenders.SLIM import SLIM_BPR_Python
from RecSys_Course_AT_PoliMi.Recommenders.GraphBased import RP3betaRecommender
from RecSys_Course_AT_PoliMi.Recommenders.KNN import UserKNNCFRecommender

In [20]:
ItemCBF = ItemKNNCBFRecommender.ItemKNNCBFRecommender(URM_train, ICM_all)
EASE_R = EASE_R_Recommender.EASE_R_Recommender(stack_matrix_train)
IALS = IALSRecommender.IALSRecommender(stack_matrix_train)
SLIM = SLIM_BPR_Python.SLIM_BPR_Python(stack_matrix_train)
RP3 = RP3betaRecommender.RP3betaRecommender(stack_matrix_train)
CF = UserKNNCFRecommender.UserKNNCFRecommender(stack_matrix_train)

In [21]:
ItemCBF.fit(topK = 7000, shrink =  10000, similarity = 'jaccard')

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22300 (58.5%), 74.31 column/sec. Elapsed time 5.00 min
Similarity column 38121 (100.0%), 73.50 column/sec. Elapsed time 8.64 min


In [22]:
#EASE_R.fit()

In [23]:
IALS.fit(num_factors = 34, alpha = 0.69)

IALSRecommender: Epoch 1 of 300. Elapsed time 1.27 min
IALSRecommender: Epoch 2 of 300. Elapsed time 2.01 min
IALSRecommender: Epoch 3 of 300. Elapsed time 2.68 min
IALSRecommender: Epoch 4 of 300. Elapsed time 3.19 min
IALSRecommender: Epoch 5 of 300. Elapsed time 3.71 min
IALSRecommender: Epoch 6 of 300. Elapsed time 4.23 min
IALSRecommender: Epoch 7 of 300. Elapsed time 5.11 min
IALSRecommender: Epoch 8 of 300. Elapsed time 5.63 min
IALSRecommender: Epoch 9 of 300. Elapsed time 6.14 min
IALSRecommender: Epoch 10 of 300. Elapsed time 6.72 min
IALSRecommender: Epoch 11 of 300. Elapsed time 7.21 min
IALSRecommender: Epoch 12 of 300. Elapsed time 7.69 min
IALSRecommender: Epoch 13 of 300. Elapsed time 8.19 min
IALSRecommender: Epoch 14 of 300. Elapsed time 8.71 min
IALSRecommender: Epoch 15 of 300. Elapsed time 9.23 min
IALSRecommender: Epoch 16 of 300. Elapsed time 9.75 min
IALSRecommender: Epoch 17 of 300. Elapsed time 10.35 min
IALSRecommender: Epoch 18 of 300. Elapsed time 10.87 min

In [24]:
#SLIM.fit(l1_norm = 0.05, l2_norm = 0.5)

TypeError: SLIM_BPR_Python.fit() got an unexpected keyword argument 'l1_norm'

In [1]:
RP3.fit(alpha = 0.775, beta = 0.495 , implicit = True, topK = 111)

NameError: name 'RP3' is not defined

In [2]:
CF.fit(topK = 432, shrink =34, similarity = 'jaccard')

NameError: name 'CF' is not defined

In [ ]:
lambda1 = 1
lambda2 = 1
lambda3 = 1
lambda4 = 1
lambda5 = 1

# Test

In [ ]:
from RecSys_Course_AT_PoliMi.Evaluation.metrics import MAP

# Initialize the MAP metric
map_metric = MAP()

In [ ]:
def recommend(user_id_array, items_to_compute=None, top_k=10):
    item_weights = np.empty([len(user_id_array), 18059])

    top_items = []  # Lista per memorizzare i top-k items per ogni utente

    for i in tqdm(range(len(user_id_array))):
        # Calcola i punteggi per ciascun modello e normalizza
        w1 = ItemCBF.recommend(user_id_array[i], items_to_compute)
        w1 /= la.norm(w1, 2)

       #w2 = EASE_R._compute_item_score(user_id_array[i], items_to_compute)
       #w2 /= la.norm(w2, 2)

       # w12 = 0.05 * w1 + 0.95 * w2

        w3 = IALS.recommend(user_id_array[i], items_to_compute)
        w3 /= la.norm(w3, 2)

        w4 = SLIM.recommend(user_id_array[i], items_to_compute)
        w4 /= la.norm(w4, 2)

        w5 = RP3.recommend(user_id_array[i], items_to_compute)
        w5 /= la.norm(w5, 2)

        w6 = CF.recommend(user_id_array[i], items_to_compute)
        w6 /= la.norm(w6, 2)

        # Combina i punteggi con i pesi specificati
        w = lambda1 * w1 + lambda2 * w3 + lambda3 * w4 + lambda4 * w5 + lambda5 * w6

        # Ordina e seleziona i primi `top_k` articoli
        top_k_indices = np.argpartition(w, -top_k)[-top_k:]
        top_k_indices = top_k_indices[np.argsort(-w[top_k_indices])]  # Ordina in ordine decrescente

        # Aggiungi gli indici dei top-k articoli per questo utente alla lista `top_items`
        top_items.append(top_k_indices)
        item_weights[i, :] = w  # Mantieni i punteggi completi se necessario per altre analisi

    return np.array(top_items)  # Restituisce solo gli indici dei top-k articoli per ogni utente


# Write on a file the recommendations

In [ ]:
trace_users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender Systems/Challenge/data_target_users_test.csv')

In [ ]:
# Specify the folder path
folder_path = '/content/drive/MyDrive/Colab Notebooks/Recommender Systems/Challenge'

# Now you can write your file to the folder
file_path = os.path.join(folder_path, 'sample_submission.csv')

with open(file_path, 'w') as file:
    file.write("user_id,item_list\n")
    # Iterate over user IDs in the 'user_id' column of the DataFrame
    for user_id in trace_users['user_id']:  # Access user IDs from the 'user_id' column
        recommended_items = recommend(user_id_array=user_id, cutoff=10)
        stringa = (' '.join(map(str, np.array(recommended_items))))
        file.write(f"{user_id}, {stringa[1: len(stringa) - 1]}\n")  # Use user_id directly

print(f"Recommendations have been written to {file_path}")